<a href="https://colab.research.google.com/github/vsidaarth/Transformer-Variants-for-HourvAhead-PV-Forecasting/blob/main/Solar_Autoformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# === Step 2: Navigate to Autoformer folder inside Drive
%cd /content/drive/MyDrive/Autoformer

# === Step 3: Add Autoformer to Python path
import sys
sys.path.append('/content/drive/MyDrive/Autoformer')

# === Step 4: Standard imports
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import plotly.express as px

# === Step 5: Import the Autoformer model
from models import Autoformer
from models.Autoformer import Model

# === Optional: Print torch version
print(torch.__version__)

Mounted at /content/drive
/content/drive/MyDrive/Autoformer
2.8.0+cu126


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import plotly.express as px



# load the data
df = pd.read_csv('/content/updated_timetable.txt')
df.columns=['datetime','irradiation_forecast','temperature_forecast','irradiation','temperature','power']
tdi = pd.to_datetime(df['datetime'], format='%d-%m-%y %H')
df.set_index(tdi, inplace=True)

# # Define date ranges to remove
# ranges_to_remove = [
#     ('2018-05-01', '2018-05-22'),
#     ('2019-09-09', '2019-09-28'),
#     ('2019-11-29', '2020-01-20'),
#     ('2013-07-23', '2013-08-26')
# ]

# # Create a boolean mask for rows to keep
# mask = pd.Series(True, index=df.index)
# for start, end in ranges_to_remove:
#     mask &= ~((df.index >= start) & (df.index <= end))

# # Apply the mask to the DataFrame
# data = df[mask]
start_date = '2014-01-01'
end_date = '2019-01-31'
# Filter the data for the specified date range
data = df[start_date:end_date]

data = data.drop(columns=['datetime','irradiation_forecast','temperature_forecast'])
data['power'] = data['power']/1000

# data['power'] = data['power']/data['power'].max()*317
# data = data[59905::]

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

# Ensure zero values are plotted correctly
# data_new.loc[data_new['power_generation'].isna(), 'power_generation'] = 0

fig = make_subplots(rows=1, cols=1)
# Plotting the Load Characteristic curve
fig.add_trace(
    go.Scatter(x=data.index, y=data['power'], name='Power Generation'),
    row=1, col=1
)

# Updating axis titles
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Power Generation (kW)", row=1, col=1)

# Updating the layout
fig.update_layout(
    height=600, width=1500, template='simple_white',
    title={
        'text': "Power Generation",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(size=14, color="Black"),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

In [ ]:
load = data.loc[:, 'power']
scaler = MinMaxScaler()
load = scaler.fit_transform(load.values.reshape(-1, 1)).flatten()

irradiation = data.loc[:, 'irradiation']
scaler_w = MinMaxScaler()
irradiation = scaler_w.fit_transform(irradiation.values.reshape(-1, 1)).flatten()

load_scaled = pd.DataFrame({
    'power_generation': load,
    'irradiation': irradiation
}, index=data.index)


shift_1 = load_scaled.shift(1,axis = 0)
shift_1.columns = ['power_generation(t-1)', 'irradiation(t-1)']

final_table = pd.concat([load_scaled['power_generation'],shift_1],axis = 1)
final_table = final_table.dropna(axis=0)

# Extracting components
final_table['hour'] = final_table.index.hour
final_table['day_of_week'] = final_table.index.weekday
final_table['month'] = final_table.index.month

# Applying cyclical encoding
final_table['hour_sin'] = np.sin(2 * np.pi * final_table['hour'] / 24)
final_table['hour_cos'] = np.cos(2 * np.pi * final_table['hour'] / 24)
final_table['month_sin'] = np.sin(2 * np.pi * final_table['month'] / 12)
final_table['month_cos'] = np.cos(2 * np.pi * final_table['month'] / 12)

# Optionally drop the original time components if they are no longer needed
final_table.drop(['hour', 'day_of_week', 'month'], axis=1, inplace=True)

# Show the modified DataFrame
final_table.head(-1)


,power_generation,power_generation(t-1),irradiation(t-1),hour_sin,hour_cos,month_sin,month_cos
datetime,,,,,,,
2014-01-01 01:00:00,0.0,0.0,0.000441,0.258819,9.659258e-01,0.5,0.866025
2014-01-01 02:00:00,0.0,0.0,0.000336,0.500000,8.660254e-01,0.5,0.866025
2014-01-01 03:00:00,0.0,0.0,0.000228,0.707107,7.071068e-01,0.5,0.866025
2014-01-01 04:00:00,0.0,0.0,0.000186,0.866025,5.000000e-01,0.5,0.866025
2014-01-01 05:00:00,0.0,0.0,0.000017,0.965926,2.588190e-01,0.5,0.866025
...,...,...,...,...,...,...,...
2019-01-31 18:00:00,0.0,0.0,0.000892,-1.000000,-1.836970e-16,0.5,0.866025
2019-01-31 19:00:00,0.0,0.0,0.000920,-0.965926,2.588190e-01,0.5,0.866025
2019-01-31 20:00:00,0.0,0.0,0.001194,-0.866025,5.000000e-01,0.5,0.866025


In [ ]:
def split_sequences_autoformer(input_par, output, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(input_par)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        if out_end_ix >= len(input_par):
            break
        seq_x = input_par[i:end_ix, :]
        seq_y = output[out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
test_set_start_date = final_table.index[-1] - pd.DateOffset(months=12)
train_set = final_table[final_table.index < test_set_start_date]
test_set = final_table[final_table.index >= test_set_start_date]

In [ ]:
NEW_SEQ_LEN  = 72
NEW_LABEL_LEN = 36
n_features = 6

n_steps_in = NEW_SEQ_LEN # new seq_len
n_steps_out = 1  # forecast horizon
input_sp = train_set.iloc[:,1:8]
output_sp = pd.DataFrame(train_set.loc[:,'power_generation'])
# Split sequences again
X_auto, y_auto = split_sequences_autoformer(input_sp.values, output_sp.values, n_steps_in, n_steps_out)

# Train-test split as before
input_train = train_set.iloc[:, 1:8]
output_train = pd.DataFrame(train_set['power_generation'])
input_test = test_set.iloc[:, 1:8]
output_test = pd.DataFrame(test_set['power_generation'])

X_train_auto, y_train_auto = split_sequences_autoformer(input_train.values, output_train.values, n_steps_in, n_steps_out)
X_test_auto, y_test_auto = split_sequences_autoformer(input_test.values, output_test.values, n_steps_in, n_steps_out)

# Convert again to torch tensors
X_train_tensor = torch.tensor(X_train_auto, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_auto, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_auto, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_auto, dtype=torch.float32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import argparse



# 1️⃣ First define the class
class Configs:
    def __init__(self):
        self.seq_len = NEW_SEQ_LEN
        self.label_len = NEW_LABEL_LEN
        self.pred_len = 1
        self.enc_in = n_features
        self.dec_in = n_features
        self.c_out = 1
        self.factor = 3
        self.d_model = 64
        self.n_heads = 2
        self.e_layers = 1
        self.d_layers = 1
        self.d_ff = 256
        self.moving_avg = 25
        self.dropout = 0.05
        self.embed = 'fixed'
        self.freq = 'h'
        self.activation = 'gelu'
        self.output_attention = False
        self.distil = True
        self.device = device

# 2️⃣ Then instantiate config
configs = Configs()
print("CHECK: configs.c_out =", configs.c_out)

# 3️⃣ Build model
model = Model(configs).to(device)

# 4️⃣ Test model output shape
with torch.no_grad():
    test_input = torch.rand((2, NEW_SEQ_LEN, n_features)).to(device)
    x_dec = test_input[:, -NEW_LABEL_LEN:, :]
    x_mark_enc = torch.zeros((2, NEW_SEQ_LEN, 4)).to(device)
    x_mark_dec = torch.zeros((2, NEW_LABEL_LEN + 1, 4)).to(device)


    output = model(test_input, x_mark_enc, x_dec, x_mark_dec)
    print("Autoformer output shape:", output.shape)


from torch.utils.data import Dataset, DataLoader

class LoadDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset
train_dataset = LoadDataset(X_train_tensor, y_train_tensor)

# Create DataLoader
BATCH_SIZE = 32  # (can increase to 64/128 depending on your RAM/GPU)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

import torch.optim as optim
label_len = NEW_LABEL_LEN
pred_len = 1
loss_fn = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        x_enc = batch_X
        x_dec = batch_X[:, -NEW_LABEL_LEN:, :]
        x_mark_enc = torch.zeros((batch_X.shape[0], NEW_SEQ_LEN, 4)).to(device)
        x_mark_dec = torch.zeros((batch_X.shape[0], NEW_LABEL_LEN + 1, 4)).to(device)

        optimizer.zero_grad()

        output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)    # [B, 1, 1]
        output = output[:, -1, 0]        # → [B]
        loss = loss_fn(output, batch_y.view(-1))  # [B]


        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.4f}")

seq_len = NEW_SEQ_LEN
model.eval()

predictions = []
true_values = []

with torch.no_grad():
    for i in range(0, len(X_test_tensor), BATCH_SIZE):
        batch_X = X_test_tensor[i:i+BATCH_SIZE].to(device)

        x_enc = batch_X
        x_dec = batch_X[:, -NEW_LABEL_LEN:, :]

        x_mark_enc = torch.zeros((batch_X.shape[0], NEW_SEQ_LEN, 4)).to(device)
        x_mark_dec = torch.zeros((batch_X.shape[0], NEW_LABEL_LEN + pred_len, 4)).to(device)

        output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
        output = output[:, -1, 0]  # [B]
        predictions.append(output.cpu().numpy())

        true_values.append(y_test_tensor[i:i+BATCH_SIZE].cpu().numpy())

predictions = np.concatenate(predictions, axis=0).reshape(-1, 1)
true_values = np.concatenate(true_values, axis=0).reshape(-1, 1)

predictions_original = scaler.inverse_transform(
    np.concatenate([predictions.reshape(-1,1), np.zeros((len(predictions), 3))], axis=1))[:,0]

true_values_original = scaler.inverse_transform(
    np.concatenate([true_values.reshape(-1,1), np.zeros((len(true_values), 3))], axis=1))[:,0]

from sklearn.metrics import mean_absolute_error, mean_squared_error
# MAE
mae = mean_absolute_error(true_values_original, predictions_original)

# RMSE
rmse = np.sqrt(mean_squared_error(true_values_original, predictions_original))

# Mean Bias Error (MBE)
mbe = np.mean(predictions_original - true_values_original)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MBE: {mbe:.2f}")



CHECK: configs.c_out = 1
Autoformer output shape: torch.Size([2, 1, 6])
Epoch [1/10], Loss: 0.1396
Epoch [2/10], Loss: 0.0518
Epoch [3/10], Loss: 0.0383
Epoch [4/10], Loss: 0.0289
Epoch [5/10], Loss: 0.0233
Epoch [6/10], Loss: 0.0191
Epoch [7/10], Loss: 0.0168
Epoch [8/10], Loss: 0.0155
Epoch [9/10], Loss: 0.0146
Epoch [10/10], Loss: 0.0142
MAE: 0.35
RMSE: 0.50
MBE: 0.15


In [ ]:
import os
import pandas as pd
import numpy as np

# === Function to save forecasts ===
def save_forecasts(truth_orig, preds_orig, test_set, model_name, filename):
    # Ensure save folder exists in Google Drive
    save_dir = "/content/drive/MyDrive/Colab Notebooks/Forecasts"
    os.makedirs(save_dir, exist_ok=True)

    # Build full output path
    out_path = os.path.join(save_dir, filename)

    # Align lengths
    n = min(len(truth_orig), len(preds_orig), len(test_set.index))

    # Build dataframe
    df = pd.DataFrame({
        "datetime": pd.to_datetime(test_set.index[:n]),
        "actual": np.asarray(truth_orig).reshape(-1)[:n],
        "forecast": np.asarray(preds_orig).reshape(-1)[:n],
        "model": model_name
    })

    # Save to CSV
    df.to_csv(out_path, index=False)
    print(f"[{model_name}] CSV saved to: {out_path}")
    return df.head()

# === Example: Save Informer results ===
save_forecasts(
    true_values_original,      # your ground truth (kW)
    predictions_original,      # your model forecasts (kW)
    test_set,                  # DataFrame with datetime index
    "Autoformer",                # model name
    "preds_Autoformer.csv"       # file name in Drive
)


[Autoformer] CSV saved to: /content/drive/MyDrive/Colab Notebooks/Forecasts/preds_Autoformer.csv


,datetime,actual,forecast,model
0,2018-01-31 23:00:00,0.0,0.205525,Autoformer
1,2018-02-01 00:00:00,0.0,0.355744,Autoformer
2,2018-02-01 01:00:00,0.0,0.366161,Autoformer
3,2018-02-01 02:00:00,0.0,0.472016,Autoformer
4,2018-02-01 03:00:00,0.0,0.376167,Autoformer


In [ ]:
# Define hyperparameter grid
seq_len_list = [96]
label_len_list = [36, 48]
d_model_list = [32, 64]
n_heads_list = [2, 4]

from itertools import product
from sklearn.metrics import mean_absolute_error, mean_squared_error

param_grid = list(product(seq_len_list, label_len_list, d_model_list, n_heads_list))
results = []

for (NEW_SEQ_LEN, NEW_LABEL_LEN, D_MODEL, N_HEADS) in param_grid:
    print(f"\n▶️ Config: seq_len={NEW_SEQ_LEN}, label_len={NEW_LABEL_LEN}, d_model={D_MODEL}, n_heads={N_HEADS}")

    # === Redefine config ===
    class Configs:
        def __init__(self):
            self.seq_len = NEW_SEQ_LEN
            self.label_len = NEW_LABEL_LEN
            self.pred_len = 1
            self.enc_in = n_features
            self.dec_in = n_features
            self.c_out = 1
            self.factor = 3
            self.d_model = D_MODEL
            self.n_heads = N_HEADS
            self.e_layers = 1
            self.d_layers = 1
            self.d_ff = 256
            self.moving_avg = 25
            self.dropout = 0.05
            self.embed = 'fixed'
            self.freq = 'h'
            self.activation = 'gelu'
            self.output_attention = False
            self.distil = True
            self.device = device

    configs = Configs()
    model = Model(configs).to(device)

    # Redefine optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = torch.nn.MSELoss()

    # Redefine dataloader
    train_dataset = LoadDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Training
    model.train()
    for epoch in range(5):  # epochs
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            x_enc = batch_X
            x_dec = batch_X[:, -NEW_LABEL_LEN:, :]
            x_mark_enc = torch.zeros((x_enc.shape[0], x_enc.shape[1], 4)).to(device)
            x_mark_dec = torch.zeros((x_dec.shape[0], x_dec.shape[1] + 1, 4)).to(device)


            optimizer.zero_grad()
            output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
            output = output[:, -1, 0]  # shape [B]
            loss = loss_fn(output, batch_y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"  Epoch {epoch+1} loss: {total_loss/len(train_loader):.4f}")

    # Evaluation
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for i in range(0, len(X_test_tensor), 32):
            batch_X = X_test_tensor[i:i+32].to(device)
            x_enc = batch_X
            x_dec = batch_X[:, -NEW_LABEL_LEN:, :]
            x_mark_enc = torch.zeros((x_enc.shape[0], x_enc.shape[1], 4)).to(device)
            x_mark_dec = torch.zeros((x_dec.shape[0], x_dec.shape[1] + 1, 4)).to(device)

            output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
            output = output[:, -1, 0]
            preds.append(output.cpu().numpy())
            trues.append(y_test_tensor[i:i+32].cpu().numpy())

    preds = np.concatenate(preds, axis=0).reshape(-1, 1)
    trues = np.concatenate(trues, axis=0).reshape(-1, 1)

    preds_orig = scaler.inverse_transform(np.concatenate([preds, np.zeros((len(preds), 3))], axis=1))[:, 0]
    trues_orig = scaler.inverse_transform(np.concatenate([trues, np.zeros((len(trues), 3))], axis=1))[:, 0]

    mae = mean_absolute_error(trues_orig, preds_orig)
    rmse = np.sqrt(mean_squared_error(trues_orig, preds_orig))
    mbe = np.mean(preds_orig - trues_orig)

    print(f"✅ MAE: {mae:.2f}, RMSE: {rmse:.2f}, MBE: {mbe:.2f}")
    results.append({
        'seq_len': NEW_SEQ_LEN,
        'label_len': NEW_LABEL_LEN,
        'd_model': D_MODEL,
        'n_heads': N_HEADS,
        'mae': mae,
        'rmse': rmse,
        'mbe': mbe
    })

# Final results table
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='mae')
print("\n🏁 Best Configurations by MAE:")
print(results_df.head())



▶️ Config: seq_len=96, label_len=36, d_model=32, n_heads=2
  Epoch 1 loss: 0.0828
  Epoch 2 loss: 0.0382
  Epoch 3 loss: 0.0258
  Epoch 4 loss: 0.0193
  Epoch 5 loss: 0.0167
✅ MAE: 0.38, RMSE: 0.52, MBE: 0.06

▶️ Config: seq_len=96, label_len=36, d_model=32, n_heads=4
  Epoch 1 loss: 0.1162
  Epoch 2 loss: 0.0448
  Epoch 3 loss: 0.0267
  Epoch 4 loss: 0.0192
  Epoch 5 loss: 0.0172
✅ MAE: 0.42, RMSE: 0.60, MBE: -0.20

▶️ Config: seq_len=96, label_len=36, d_model=64, n_heads=2
  Epoch 1 loss: 0.0804
  Epoch 2 loss: 0.0288
  Epoch 3 loss: 0.0181
  Epoch 4 loss: 0.0158
  Epoch 5 loss: 0.0146
✅ MAE: 0.36, RMSE: 0.51, MBE: -0.03

▶️ Config: seq_len=96, label_len=36, d_model=64, n_heads=4
  Epoch 1 loss: 0.0745
  Epoch 2 loss: 0.0265
  Epoch 3 loss: 0.0180
  Epoch 4 loss: 0.0159
  Epoch 5 loss: 0.0149
✅ MAE: 0.38, RMSE: 0.56, MBE: -0.12

▶️ Config: seq_len=96, label_len=48, d_model=32, n_heads=2
  Epoch 1 loss: 0.0820
  Epoch 2 loss: 0.0420
  Epoch 3 loss: 0.0288
  Epoch 4 loss: 0.0221
  Epo

In [ ]:
from scipy.stats import norm
import numpy as np



def compute_crps(actual, lower, upper):
    """
    Computes CRPS using a quantile-based approximation.
    """
    if actual < lower:
        return (upper - lower) + ((lower - actual) ** 2) / (upper - lower)
    elif actual > upper:
        return (upper - lower) + ((actual - upper) ** 2) / (upper - lower)
    else:
        return (upper - lower) / 2


def compute_winkler_score(y_true, y_lower, y_upper, alpha=0.1):
    """
    Computes the Winkler Score.

    Args:
        y_true (float): The actual observed value.
        y_lower (float): The predicted lower quantile.
        y_upper (float): The predicted upper quantile.
        alpha (float): The significance level (default 0.1 for a 90% prediction interval).

    Returns:
        float: Winkler score for the given forecast.
    """
    interval_width = y_upper - y_lower

    if y_true < y_lower:
        return interval_width + (2 / alpha) * (y_lower - y_true)
    elif y_true > y_upper:
        return interval_width + (2 / alpha) * (y_true - y_upper)
    else:
        return interval_width

# Initialize Arrays for Metrics
y_lowers_auto = np.empty(len(predictions_original))
y_uppers_auto = np.empty(len(predictions_original))
err_i_auto = np.empty(len(predictions_original))
alpha_t_i_auto = np.empty(len(predictions_original))
window_i_auto = np.empty(len(predictions_original))
res_cal_auto = np.empty(len(predictions_original))
alpha_t_auto = 0.1
gamma_auto = 0.01

# Initialize Scores
crps_auto = np.zeros(len(predictions_original))
winkler_auto = np.zeros(len(predictions_original))

for i in range(1, len(predictions_original)):
    res_cal_auto = np.abs(predictions_original - true_values_original)

    if alpha_t_auto >= 1:
        y_lowers_auto[i], y_uppers_auto[i] = 0, 0
        err_auto = 1
    elif alpha_t_auto <= 0:
        y_lowers_auto[i], y_uppers_auto[i] = 0, 0
        err_auto = 0
        alpha_t_auto = 0.1
    else:
        window_auto = np.quantile(res_cal_auto, (1 - alpha_t_auto))
        y_lowers_auto[i] = predictions_original[i] - window_auto
        y_uppers_auto[i] = predictions_original[i] + window_auto
        err_auto = 1 - float((y_lowers_auto[i] <= true_values_original[i]) and (true_values_original[i] <= y_uppers_auto[i]))

    # Update alpha adaptively
    alpha_t_auto = alpha_t_auto + gamma_auto * (alpha_t_auto - err_auto)
    if alpha_t_auto < 0 or alpha_t_auto > 0.3:
        alpha_t_auto = 0.1

    # Store values
    alpha_t_i_auto[i] = alpha_t_auto
    err_i_auto[i] = err_auto
    window_i_auto[i] = window_auto

    crps_auto[i] = compute_crps(true_values_original[i], y_lowers_auto[i], y_uppers_auto[i])
    winkler_auto[i] = compute_winkler_score(true_values_original[i], y_lowers_auto[i], y_uppers_auto[i], alpha=0.1)

# Clip bounds
y_lowers_auto[y_lowers_auto < 0] = 0
y_uppers_auto[y_uppers_auto > max(true_values_original)] = max(true_values_original)
y_lowers_auto = y_lowers_auto.reshape((predictions_original.shape[0], 1))
y_uppers_auto = y_uppers_auto.reshape((predictions_original.shape[0], 1))

# Metrics
interval_widths_auto = y_uppers_auto - y_lowers_auto
mean_interval_width_auto = np.mean(interval_widths_auto)

covered_auto = (true_values_original >= y_lowers_auto) & (true_values_original <= y_uppers_auto)
coverage_percentage_auto = np.mean(covered_auto) * 100

mean_crps_auto = np.mean(crps_auto)
mean_winkler_auto = np.mean(winkler_auto)

print(f"Mean Interval Width: {mean_interval_width_auto}")
print(f"Coverage Percentage: {coverage_percentage_auto}%")
print(f"Mean CRPS: {mean_crps_auto}")
print(f"Mean Winkler Score: {mean_winkler_auto}")


Mean Interval Width: 1.4162834016274755
Coverage Percentage: 59.23071862369695%
Mean CRPS: 1.063596956588421
Mean Winkler Score: 2.629264204983546


In [ ]:
px.line(alpha_t_i_auto)